In [1]:
#!pip install azure-ai-ml
#!pip install azure-identity
#!pip install azure-ai-projects

In [2]:
import asyncio
from typing import Annotated

from azure.identity.aio import DefaultAzureCredential

from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.functions import kernel_function

"""
The following sample demonstrates how to create an Azure AI agent that answers
questions about a sample menu using a Semantic Kernel Plugin.
"""
import logging
import json
import textwrap
import re

from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# NOTE: This is all that is required to enable logging.
# Set the desired level to INFO, DEBUG, etc.
logging.basicConfig(level=logging.WARNING)

In [9]:
# Define a sample plugin for the sample
class MenuPlugin:
    """A sample Menu Plugin used for the concept sample."""

    @kernel_function(description="Provides a list of specials from the menu.")
    def get_specials(self) -> Annotated[str, "Returns the specials from the menu."]:
        return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

    @kernel_function(description="Provides the price of the requested menu item.")
    def get_item_price(
        self, menu_item: Annotated[str, "The name of the menu item."]
    ) -> Annotated[str, "Returns the price of the menu item."]:
        return "$9.99"


# Simulate a conversation with the agent
USER_INPUTS = [
    "Hello",
    "What is the special soup?",
    "How much does that cost?",
    "Thank you",
]

ai_agent_settings = AzureAIAgentSettings(model_deployment_name="gpt-4o-mini")

async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
    ) as client,
):
    
    # 1. Create an agent on the Azure AI agent service
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        #model="gpt-4o-mini",
        name="Host",
        instructions="Answer questions about the menu.",
    )

    # 2. Create a Semantic Kernel agent for the Azure AI agent
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[MenuPlugin()],  # Add the plugin to the agent
    )

    # 3. Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread = None

    try:
        for user_input in USER_INPUTS:
            print(f"# User: {user_input}")
            # 4. Invoke the agent for the specified thread for response
            async for response in agent.invoke(
                messages=user_input,
                thread=thread,
            ):
                print(f"# {response.name}: {response}")
                thread = response.thread
    finally:
        # 5. Cleanup: Delete the thread and agent
        await thread.delete() if thread else None
        #await client.agents.delete_agent(agent.id)

    """
    Sample Output:
    # User: Hello
    # Agent: Hello! How can I assist you today?
    # User: What is the special soup?
    # ...
    """

# User: Hello
# Host: Hello! How can I assist you today?
# User: What is the special soup?
# Host: The special soup is Clam Chowder. Would you like to know more about the other specials on the menu?
# User: How much does that cost?
# Host: The Clam Chowder costs $9.99. Would you like to know anything else?
# User: Thank you
# Host: You're welcome! If you have any more questions or need assistance in the future, feel free to ask. Have a great day!
